In [2]:
# Import the required modules
%pylab inline 
import cv2
from IPython.display import clear_output

import pickle as cPickle
import gzip

import numpy as np
%pylab inline 
import matplotlib.pyplot as pl

# from PIL import Image
# import PIL.ImageOps 

Populating the interactive namespace from numpy and matplotlib
Populating the interactive namespace from numpy and matplotlib


In [3]:
dirPath = 'ASL/signs/'
import os
signs = ''
trainingX = []
trainingY = []
training = []
for fl in os.listdir(dirPath):
    fname = dirPath+fl
    base=os.path.basename(fname)
    symbol = os.path.splitext(base)[0]  
    im = cv2.imread(fname)
#     im = cv2.imread(fname, cv2.CV_LOAD_IMAGE_GRAYSCALE)
    im = im.invert(im)
#     img = mpimg.imread(fname)
    im = im.convert('1')
    im = im.crop((0, 0, 250, 250))
    im = im.resize((100,100),Image.ANTIALIAS)
    (width, height) = im.size
    imgB = np.asarray(im).reshape(-1)
    trainingX.append(imgB)
    trainingY.append(symbol)
    signs += symbol
    pass
signs = sorted(signs)
# signs.index('z')
# print(width,height)
training.append(trainingX)
training.append(trainingY)
print(trainingX[0])

AttributeError: 'numpy.ndarray' object has no attribute 'invert'

In [ ]:
def vectorized_result(j,size):
    e = np.zeros((26, size))
#     print('j:',j)
#     print('size',size)
    cnt=0
    for i in j:
        idx = signs.index(i)
        e[idx][cnt] = 1.0
        cnt+=1
    return e

In [ ]:
Xdisplay = np.reshape(training[0][1],(width,height))
y = vectorized_result(training[1][0:3],3)
# pl.imshow(Xdisplay,cmap='Greys')
X = np.float32(training[0][0:3])

In [ ]:
class Neural_Network(object):
    def __init__(self,width,height):        
        self.inputLayerSize = width*height
        self.outputLayerSize = 26
        self.hiddenLayerSize = 30

        self.W1 = np.random.randn(self.inputLayerSize,self.hiddenLayerSize)
        self.W2 = np.random.randn(self.hiddenLayerSize,self.outputLayerSize)
        
    def forward(self, X):
        self.z2 = np.dot(X, self.W1)
        self.a2 = self.sigmoid(self.z2)
        self.z3 = np.dot(self.a2, self.W2)
        yHat = self.sigmoid(self.z3) 
        return yHat
        
    def sigmoid(self, z):
        return 1/(1+np.exp(-z))
    
    def sigmoidPrime(self,z):
        return np.exp(-z)/((1+np.exp(-z))**2)
    
    def costFunction(self, X, y):
        self.yHat = self.forward(X)
        J = 0.5*sum((y-self.yHat)**2)
        return J
        
    def costFunctionPrime(self, X, y):
        self.yHat = self.forward(X)
        delta3 = np.multiply(-(y-self.yHat), self.sigmoidPrime(self.z3))
        dJdW2 = np.dot(self.a2.T, delta3)
        delta2 = np.float32(np.dot(delta3, self.W2.T)*self.sigmoidPrime(self.z2))
        dJdW1 = np.dot(X.T, delta2)
        return dJdW1, dJdW2
    
    def getParams(self):
        params = np.concatenate((self.W1.ravel(), self.W2.ravel()))
        return params
    
    def setParams(self, params):
        W1_start = 0
        W1_end = self.hiddenLayerSize * self.inputLayerSize
        self.W1 = np.reshape(params[W1_start:W1_end], (self.inputLayerSize , self.hiddenLayerSize))
        W2_end = W1_end + self.hiddenLayerSize*self.outputLayerSize
        self.W2 = np.reshape(params[W1_end:W2_end], (self.hiddenLayerSize, self.outputLayerSize))
        
    def computeGradients(self, X, y):
        dJdW1, dJdW2 = self.costFunctionPrime(X, y)
        return np.concatenate((dJdW1.ravel(), dJdW2.ravel()))

In [ ]:
NN = Neural_Network(width,height)

In [ ]:
costarr=[]
for i in range(0,10):
    for j in range(0,len(training[0]),2):
#     for j in range(0,10,2): # len(training[0]),2):
#         print(j)
        y = vectorized_result(training[1][j:j+2],2)
#         print('y',y.shape)
        X = np.float32(training[0][j:j+2])
#         X = X.reshape(X.shape[0],1)
#         print('X',X.shape)
        cost = NN.costFunction(X,y.T)
        costarr.append(cost)
        dJdW1, dJdW2 = NN.costFunctionPrime(X,y.T)
        scalar = 2
        NN.W1 = NN.W1 - scalar*dJdW1
        NN.W2 = NN.W2 - scalar*dJdW2
# print(costarr)

In [ ]:
plot(costarr)
grid(1)
xlabel('Iterations')
ylabel('Cost')

In [ ]:
X = np.float32(training[0][0:3])
result=NN.forward(X)
result=np.around(result, decimals = 2)
print(result)
yarr=training[1][0:3]
res=[np.argmax(i) for i in  result]
print([signs[sign] for sign in res])
print('yarr',yarr)
sum(int(x == y) for (x, y) in list(zip(res,yarr)))

In [ ]:
try:
#     cap = cv2.VideoCapture(0)
    cap = cv2.VideoCapture(-1)
    while(1):
        # Capture frame-by-frame
        ret, frame = cap.read()
#        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.cvtColor(frame, cv2.THRESH_BINARY)
    
        imshow(frame)#cmap='Greys')
        show()
        # Display the frame until new frame is available
        clear_output(wait=True)
except KeyboardInterrupt:
    # Release the Video Device
    cap.release()
    # Message to be displayed after releasing the device
    print("Released Video Resource")